# **COURSE DATABASE**

In [1]:
%load_ext sql
import pandas as pd
import sqlite3

## **1. Create database**

In [2]:
%sql sqlite:///CourseData.db
conn = sqlite3.connect('CourseData.db')

## **2. Create tables**

In [3]:
%%sql

-- Program table
DROP TABLE IF EXISTS PROGRAM;
CREATE TABLE PROGRAM (
    Program_id INTEGER PRIMARY KEY,
    Program_code TEXT NOT NULL,
    Program_name TEXT NOT NULL
);

-- Location table
DROP TABLE IF EXISTS LOCATION;
CREATE TABLE LOCATION (
    Location_id INTEGER PRIMARY KEY,
    Location TEXT
);


-- Course table
DROP TABLE IF EXISTS COURSE;
CREATE TABLE COURSE (
    Course_id INTEGER PRIMARY KEY,
    Catalog_year TEXT NOT NULL,
    Program_id INTEGER NOT NULL,
    CatalogID TEXT NOT NULL,
    Course_title TEXT NOT NULL, 
    Description TEXT,
    Credits REAL,
    Attributes TEXT,
    Prerequisites TEXT,
    Corequisites TEXT,
    Fees TEXT,
    FOREIGN KEY (Program_id) REFERENCES PROGRAM(Program_id)
);


-- Instructor table
DROP TABLE IF EXISTS INSTRUCTOR;
CREATE TABLE INSTRUCTOR (
    Instructor_id INTEGER PRIMARY KEY,
    Name TEXT NOT NULL,
    Phone INTEGER,
    Email TEXT
);

-- Class table
DROP TABLE IF EXISTS CLASS;
CREATE TABLE CLASS (
    Class_id INTEGER PRIMARY KEY,
    Course_id INTEGER,
    Section TEXT NOT NULL,
    Instructor_id INTEGER,
    CatalogID TEXT NOT NULL,
    Course_title TEXT NOT NULL, 
    Term TEXT NOT NULL, 
    crn INTEGER NOT NULL,
    Cap INTEGER NOT NULL,
    Act INTEGER NOT NULL,
    Rem INTEGER NOT NULL,
    Timecode TEXT,
    FOREIGN KEY (Course_id) REFERENCES COURSE(Course_id)
    FOREIGN KEY (Instructor_id) REFERENCES INSTRUCTOR(Instuctor_id)
);

-- Meeting table
DROP TABLE IF EXISTS MEETING;
CREATE TABLE MEETING (
    Meeting_id INTEGER PRIMARY KEY,
    Class_id INTEGER NOT NULL,
    Location_id INTEGER NOT NULL,
    Start TEXT NOT NULL,
    End TEXT NOT NULL,
    Day TEXT NOT NULL,
    FOREIGN KEY (Class_id) REFERENCES CLASS(Class_id),
    FOREIGN KEY (Location_id) REFERENCES LOCATION(Location_id)
);

-- A conversion table for matching the term to the corresponding catalog year
-- Catalogs are not available for the first several years
DROP TABLE IF EXISTS TERM_CATALOG_YEAR;
CREATE TABLE TERM_CATALOG_YEAR (
    CatalogYear TEXT NOT NULL,
    Term TEXT NOT NULL
);

INSERT INTO TERM_CATALOG_YEAR (Term, CatalogYear) VALUES 
('Fall2017','2017_2018'),('Winter2018','2017_2018'),('Spring2018','2017_2018'),('Summer2018','2017_2018'),('Fall2018','2018_2019'),
('Winter2019','2018_2019'),('Spring2019','2018_2019');


 * sqlite:///CourseData.db
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
7 rows affected.


[]

## **3. Extract data from data source**

In [4]:
# Catalog Data
catalog_years = ['2017_2018', '2018_2019']

for cat_year in catalog_years:
    filepath = 'SourceData/Catalogs/CourseCatalog'+cat_year+'.csv'
    data = pd.read_csv(filepath)
    data['cat_year'] = cat_year
    data.to_sql('IMPORT_CATALOG_COURSES',conn,if_exists='append',index=False)
    


In [5]:
# Course Offering and Course Meeting Data
terms = ['Fall2014','Fall2015','Fall2016','Fall2017','Fall2018',
         'Spring2015','Spring2016','Spring2017','Spring2018','Spring2019',
         'SpringBreak2017',
         'Summer2015','Summer2016','Summer2017','Summer2018',
         'Winter2015','Winter2016','Winter2017','Winter2018']
for term in terms:
    filepath = 'SourceData/'+term+'/courses.csv'
    data = pd.read_csv(filepath)
    data.to_sql('IMPORT_COURSE_OFFERINGS',conn,if_exists='append',index=False) 
    
    filepath = 'SourceData/'+term+'/course_meetings.csv'
    data = pd.read_csv(filepath)
    data.to_sql('IMPORT_COURSE_MEETINGS',conn,if_exists='append',index=False)

In [6]:
%%sql
-- Record Counts for Catalog Courses
SELECT 
    (SELECT Count(*) FROM IMPORT_CATALOG_COURSES) as 'RawCount',
    (SELECT Count(*) FROM (SELECT DISTINCT * FROM IMPORT_CATALOG_COURSES)) as 'DistinctCount';

 * sqlite:///CourseData.db
Done.


RawCount,DistinctCount
4440,4440


In [7]:
%%sql
-- Record Counts for Course Offerings
SELECT 
    (SELECT Count(*) FROM IMPORT_COURSE_OFFERINGS) as 'RawCount',
    (SELECT Count(*) FROM (SELECT DISTINCT * FROM IMPORT_COURSE_OFFERINGS)) as 'DistinctCount';

 * sqlite:///CourseData.db
Done.


RawCount,DistinctCount
15937,15937


In [8]:
%%sql 
-- Record Counts for Catalog Meetings
SELECT 
    (SELECT Count(*) FROM IMPORT_COURSE_MEETINGS) as 'RawCount',
    (SELECT Count(*) FROM (SELECT DISTINCT * FROM IMPORT_COURSE_MEETINGS)) as 'DistinctCount';

 * sqlite:///CourseData.db
Done.


RawCount,DistinctCount
284907,284847


In [1]:
%%sql 
SELECT term,crn,location, day, start 
FROM IMPORT_COURSE_MEETINGS
GROUP BY term,crn,location, day, start
HAVING COUNT(*)>1
LIMIT 5

UsageError: Cell magic `%%sql` not found.


## **4. Load data into *CourseData* database**

In [10]:
%%sql
DELETE FROM PROGRAM;

INSERT INTO PROGRAM (Program_code, Program_name)
SELECT DISTINCT program_code, program_name
    FROM IMPORT_CATALOG_COURSES
    ORDER BY Program_code;


 * sqlite:///CourseData.db
0 rows affected.
83 rows affected.


[]

In [11]:
%%sql 
SELECT *
FROM PROGRAM 
LIMIT 5;

 * sqlite:///CourseData.db
Done.


Program_id,Program_code,Program_name
1,AC,Accounting
2,AE,Applied Ethics
3,AH,Art History
4,AN,Asian Studies
5,AR,Arabic


In [12]:
%%sql

INSERT INTO LOCATION (Location)
SELECT DISTINCT location
FROM IMPORT_COURSE_MEETINGS
ORDER BY Location;


 * sqlite:///CourseData.db
207 rows affected.


[]

In [13]:
%%sql 
SELECT *
FROM LOCATION 
LIMIT 5;

 * sqlite:///CourseData.db
Done.


Location_id,Location
1,BCC 200
2,BD
3,BH
4,BH BY ARR
5,BLM 112


In [14]:
%%sql
DELETE FROM COURSE;

INSERT INTO COURSE (Catalog_year, Program_id, CatalogID, Course_title, Description, Credits, Attributes, Prerequisites, Corequisites, Fees)
SELECT DISTINCT cat_year, program_id, catalog_id, course_title, description, credits, attributes, prereqs, coreqs, fees
    FROM IMPORT_CATALOG_COURSES
    JOIN PROGRAM ON (IMPORT_CATALOG_COURSES.program_code = PROGRAM.Program_code);



 * sqlite:///CourseData.db
0 rows affected.
4440 rows affected.


[]

In [15]:
%%sql 
SELECT *
    FROM COURSE 
    LIMIT 1;

 * sqlite:///CourseData.db
Done.


Course_id,Catalog_year,Program_id,CatalogID,Course_title,Description,Credits,Attributes,Prerequisites,Corequisites,Fees
1,2017_2018,4,AN 0301,Independent Study,Students undertake an individualized program of study in consultation with a director from the Asian studies faculty.,1-3 Credits,None,None,None,None


In [16]:
%%sql
DELETE FROM INSTRUCTOR;

INSERT INTO INSTRUCTOR (Name)
SELECT DISTINCT primary_instructor
    FROM import_course_offerings 
    WHERE primary_instructor <> 'TBA' AND primary_instructor NOT LIKE '%/%'
    ORDER BY primary_instructor

 * sqlite:///CourseData.db
0 rows affected.
1095 rows affected.


[]

In [17]:
%%sql 
SELECT *
    FROM INSTRUCTOR 
    LIMIT 5;

 * sqlite:///CourseData.db
Done.


Instructor_id,Name,Phone,Email
1,Aaron K. Perkus,None,None
2,Aaron Quinn Weinstein,None,None
3,Aaron R. Van Dyke,None,None
4,Abdel Illah A. Douda,None,None
5,Adam E. Rugg,None,None


In [18]:
%%sql
SELECT *
    FROM IMPORT_COURSE_OFFERINGS
    LIMIT 1

 * sqlite:///CourseData.db
Done.


term,crn,catalog_id,section,credits,title,meetings,timecodes,primary_instructor,cap,act,rem
Fall2014,70384,AC 0011,C01,3.000,Introduction to Financial Accounting,"[{'days': 'TF', 'times': '0800am-0915am', 'dates': '09/02-12/08', 'location': 'DSB 105'}]",['TF 0800am-0915am 09/02-12/08 DSB 105'],Michael P. Coyne,0,31,-31


In [19]:
%%sql
DELETE FROM CLASS;

INSERT INTO CLASS (Course_id, Section, Instructor_id, CatalogID, Course_title, Term, crn, Cap, Act, Rem, Timecode)
SELECT DISTINCT Course_id , section, INSTRUCTOR.Instructor_id, IMPORT_COURSE_OFFERINGS.catalog_id, IMPORT_COURSE_OFFERINGS.title, term, crn, cap, act, rem, timecodes
FROM IMPORT_COURSE_OFFERINGS 
    LEFT JOIN INSTRUCTOR ON (IMPORT_COURSE_OFFERINGS.primary_instructor=INSTRUCTOR.Name)
    LEFT JOIN TERM_CATALOG_YEAR USING (Term)
    LEFT JOIN COURSE ON (IMPORT_COURSE_OFFERINGS.catalog_id = COURSE.CatalogID AND COURSE.Catalog_year = TERM_CATALOG_YEAR.CatalogYear);

 * sqlite:///CourseData.db
0 rows affected.
15937 rows affected.


[]

In [20]:
%%sql 
SELECT *
    FROM CLASS 
    ORDER BY Class_id
    LIMIT 2;

 * sqlite:///CourseData.db
Done.


Class_id,Course_id,Section,Instructor_id,CatalogID,Course_title,Term,crn,Cap,Act,Rem,Timecode
1,None,C01,744,AC 0011,Introduction to Financial Accounting,Fall2014,70384,0,31,-31,['TF 0800am-0915am 09/02-12/08 DSB 105']
2,None,C02,744,AC 0011,Introduction to Financial Accounting,Fall2014,70385,0,31,-31,['TF 0930am-1045am 09/02-12/08 DSB 105']
3,None,C03,744,AC 0011,Introduction to Financial Accounting,Fall2014,70382,0,31,-31,['TF 1230pm-0145pm 09/02-12/08 DSB 105']
4,None,C04,871,AC 0011,Introduction to Financial Accounting,Fall2014,70291,0,29,-29,['MR 1100am-1215pm 09/02-12/08 DSB 111']
5,None,C05,871,AC 0011,Introduction to Financial Accounting,Fall2014,70350,0,30,-30,['MR 1230pm-0145pm 09/02-12/08 DSB 111']
6,None,C06,871,AC 0011,Introduction to Financial Accounting,Fall2014,70381,0,31,-31,['MR 0330pm-0445pm 09/02-12/08 DSB 110A']
7,None,G,829,AC 0011,Introduction to Financial Accounting,Fall2014,70383,30,31,-1,['MR 0930am-1045am 09/02-12/08 DSB 111']
8,None,H,447,AC 0011,Introduction to Financial Accounting,Fall2014,70391,30,32,-2,['TF 0930am-1045am 09/02-12/08 DSB 110A']
9,None,I,447,AC 0011,Introduction to Financial Accounting,Fall2014,71105,30,33,-3,['TF 1100am-1215pm 09/02-12/08 DSB 110A']
10,None,J,447,AC 0011,Introduction to Financial Accounting,Fall2014,71123,30,32,-2,['TF 0200pm-0315pm 09/02-12/08 DSB 105']


In [21]:
%%sql
DELETE FROM MEETING;

INSERT INTO MEETING (Class_id, Location_id, Start, End, Day)
SELECT DISTINCT Class_id, Location_id, start, end, day 
FROM IMPORT_COURSE_MEETINGS 
    JOIN CLASS USING (term, crn)
    LEFT JOIN LOCATION ON (LOCATION.Location = import_course_meetings.location)


 * sqlite:///CourseData.db
0 rows affected.
284847 rows affected.


[]

In [22]:
%%sql 
SELECT *
    FROM MEETING 
    ORDER BY Class_id
    LIMIT 2;

 * sqlite:///CourseData.db
Done.


Meeting_id,Class_id,Location_id,Start,End,Day
1,1,99,2014-09-02T08:00:00,2014-09-02T09:15:00,T
2,1,99,2014-09-05T08:00:00,2014-09-05T09:15:00,F
3,1,99,2014-09-09T08:00:00,2014-09-09T09:15:00,T
4,1,99,2014-09-12T08:00:00,2014-09-12T09:15:00,F
5,1,99,2014-09-16T08:00:00,2014-09-16T09:15:00,T
6,1,99,2014-09-19T08:00:00,2014-09-19T09:15:00,F
7,1,99,2014-09-23T08:00:00,2014-09-23T09:15:00,T
8,1,99,2014-09-26T08:00:00,2014-09-26T09:15:00,F
9,1,99,2014-09-30T08:00:00,2014-09-30T09:15:00,T
10,1,99,2014-10-03T08:00:00,2014-10-03T09:15:00,F


## **5. Delete temporary tables**

%%sql
DELETE FROM IMPORT_CATALOG_COURSES;
DELETE FROM IMPORT_COURSE_OFFERINGS;
DELETE FROM IMPORT_COURSE_MEETINGS;

%%sql
DROP TABLE IMPORT_CATALOG_COURSES;
DROP TABLE IMPORT_COURSE_OFFERINGS;
DROP TABLE IMPORT_COURSE_MEETINGS;

Command ***vacuum*** cleans the main database by copying its contents to a temporary database file and reloading the original database file from the copy. This eliminates free pages, aligns table data to be contiguous, and otherwise cleans up the database file structure.

%%sql
vacuum;